In [1]:
from python_profiling.time_profiling.time_profiling_decorators import CallGraphTimeProfilerDecorator
from python_profiling.python_profiling_configs import StorageConfig
from python_profiling.python_profiling_enums import SerializerStrategy

storages = StorageConfig(serializers_strategies=[SerializerStrategy.JSON],
                         file_paths=['data.json'])

@CallGraphTimeProfilerDecorator(sort_key='11',storages=storages)
def hello(name):
    for i in range(10):
        1 / 0

result = hello(name='bob')
print(result)

ValidationError: 1 validation error for CallGraphTimeProfiler
  Value error, Invalid sort key, valid options: [time, cumulative, calls, name, file, line] [type=value_error, input_value={'sort_key': '11', 'func_filter': '', 'top_n': 10}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/value_error